## Web Assembly

In [1]:
#include <base/net/HTTPSRequest.h>
#include <base/string/Format.h>

In [2]:
using namespace base;

In [3]:
String url = "https://github.com/rene-fonseca/docker-webassembly/raw/master/wasmtime/hello";

In [4]:
auto response = HTTPSRequest::load(url)

In [5]:
#include <base/string/FormatOutputStream.h>

In [6]:
response.getSecond()

"application/octet-stream"

In [7]:
#include <base/webassembly/WebAssembly.h>

In [8]:
response.getFirst().getLength()

31259

In [9]:
WebAssembly wasm;

In [10]:
wasm.load(response.getFirst())

true

In [11]:
fout << "IMPORTS:" << EOL;
for (const auto& s : wasm.getImports()) {
  fout << "  " << "[" << s.index << "] ";
  switch (s.externType) {
  case WebAssembly::EXTERN_FUNCTION:
    fout << "FUNCTION " << WebAssembly::toString(s) << ENDL;
    break;
  case WebAssembly::EXTERN_GLOBAL:
    fout << "GLOBAL " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_TABLE:
    fout << "TABLE " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_MEMORY:
    fout << "MEMORY " << s.name << ENDL;
    break;
  default:
    fout << "?" << " " << s.name << ENDL;
  }
}

IMPORTS:
  [0] FUNCTION [i32] wasi_unstable!fd_prestat_get(i32, i32)
  [1] FUNCTION [i32] wasi_unstable!fd_prestat_dir_name(i32, i32, i32)
  [2] FUNCTION [] wasi_unstable!proc_exit(i32)
  [3] FUNCTION [i32] wasi_unstable!fd_fdstat_get(i32, i32)
  [4] FUNCTION [i32] wasi_unstable!fd_close(i32)
  [5] FUNCTION [i32] wasi_unstable!args_sizes_get(i32, i32)
  [6] FUNCTION [i32] wasi_unstable!args_get(i32, i32)
  [7] FUNCTION [i32] wasi_unstable!fd_write(i32, i32, i32, i32)
  [8] FUNCTION [i32] wasi_unstable!fd_seek(i32, i64, i32, i32)


In [12]:
wasm.makeInstance(true)

true

In [13]:
fout << "EXPORTS:" << EOL;
for (const auto& s : wasm.getExports()) {
  fout << "  " << "[" << s.index << "] ";
  switch (s.externType) {
  case WebAssembly::EXTERN_FUNCTION:
    fout << "FUNCTION " << WebAssembly::toString(s) << ENDL;
    break;
  case WebAssembly::EXTERN_GLOBAL:
    fout << "GLOBAL " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_TABLE:
    fout << "TABLE " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_MEMORY:
    fout << "MEMORY SIZE=" << s.memorySize << ENDL;
    break;
  default:
    fout << "?" << " " << s.name << ENDL;
  }
}

EXPORTS:
  [0] MEMORY SIZE=131072
  [1] FUNCTION [] _start()


In [14]:
wasm.call("_start")

WebAssemble: calling: _start()
wasi_unstable!fd_prestat_get(i32: 3 = 0x3, i32: 70808 = 0x11498) -> (i32: 0 = 0x0) INVOKES=1


Error: 